In [4]:
!pip install openpyxl



   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [13]:
import os
import re
import json
from typing import Dict, List, Tuple, Optional, Any


import numpy as np
import pandas as pd

# -------------------------
# 0) 설정
# -------------------------
employee_df = pd.read_excel("맥킨지 모델 적용 정량 데이터 변수 리스트.xlsx", sheet_name="맥킨지X정량데이터 매핑 테이블")


EXCEL_PATH = "맥킨지 모델 적용 정량 데이터 변수 리스트.xlsx"
SHEET_NAME = "맥킨지X정량데이터 매핑 테이블"

DRIVERS = [
    "Trust",
    "Communication",
    "Innovative Thinking",
    "Decision Making",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

# 점수 스케일: 사내 데이터가 1~5 리커트라고 가정 (시트 상단에 *리커드 척도(5점) 표기)
SCORE_MIN, SCORE_MAX = 1.0, 5.0


# -------------------------
# 1) 유틸: 정규화/클리닝
# -------------------------
def is_blank(x) -> bool:
    return (pd.isna(x)) or (str(x).strip() == "")

def normalize_1_to_5(x: float) -> float:
    """1~5 스케일로 들어온 값의 안전 처리"""
    if x is None or pd.isna(x):
        return np.nan
    x = float(x)
    # 혹시 0~1이면 1~5로 올려줌
    if 0.0 <= x <= 1.0:
        return 1.0 + 4.0 * x
    # 혹시 0~100이면 1~5로 스케일링(대충)
    if 0.0 <= x <= 100.0:
        return 1.0 + 4.0 * (x / 100.0)
    return min(max(x, SCORE_MIN), SCORE_MAX)

def safe_mean(values: List[float]) -> Optional[float]:
    vals = [v for v in values if v is not None and not pd.isna(v)]
    if not vals:
        return None
    return float(np.mean(vals))

def clean_cell(x) -> str:
    if pd.isna(x):
        return ""
    return str(x).strip()


# -------------------------
# 2) 3번째 시트 "구간 테이블" 파서
#   - ① 인사기록카드 매핑
#   - ② KPI 매핑
#   - ③ 인적성 매핑
# -------------------------
def find_section_row(df: pd.DataFrame, keyword: str) -> Optional[int]:
    """df 전체에서 keyword가 포함된 행 인덱스를 찾아 반환"""
    for i in range(len(df)):
        row_text = " ".join([clean_cell(v) for v in df.iloc[i].tolist()])
        if keyword in row_text:
            return i
    return None

def parse_matrix_table(
    df: pd.DataFrame,
    header_row: int,
    start_row: int,
    driver_col_idx: int = 1
) -> Dict[str, List[str]]:
    """
    header_row: 컬럼명이 있는 행(예: '구분', '기본 인적사항', ...)
    start_row: 실제 드라이버 행 시작(Trust가 있는 행)
    driver_col_idx: 드라이버명이 들어있는 열 인덱스 (보통 Unnamed:1)
    반환: {Driver: [변수명1, 변수명2, ...]}
    """
    headers = [clean_cell(x) for x in df.iloc[header_row].tolist()]
    out: Dict[str, List[str]] = {}

    for r in range(start_row, len(df)):
        driver = clean_cell(df.iloc[r, driver_col_idx])
        if is_blank(driver):
            break
        if driver not in DRIVERS:
            # 혹시 표 중간에 다른 텍스트가 끼면 스킵
            continue

        vars_for_driver = []
        for c in range(len(headers)):
            if c == driver_col_idx:
                continue
            cell = clean_cell(df.iloc[r, c])

            # 표에서는 의미없는 '-' 또는 '–' 로 비포함 표시
            if cell in ["-", "–", "—", ""]:
                continue

            # KPI 표에는 '핵심 KPI' 같은 라벨이 있을 수 있음 -> 변수명으로 간주
            # 인적성 표에는 '●'가 있을 수 있음 -> 이 경우 헤더를 변수명으로 채택
            if cell == "●":
                var_name = headers[c]
            else:
                var_name = cell

            if var_name and var_name not in ["구분", "KPI 항목", "인적성 영역"]:
                vars_for_driver.append(var_name)

        out[driver] = vars_for_driver

    # 중복 제거
    out = {k: sorted(list(set(v))) for k, v in out.items()}
    return out

def load_driver_variable_mapping(excel_path: str) -> Dict[str, Dict[str, List[str]]]:
    """
    반환 구조:
    {
      "hr_card": {Driver: [변수...]},
      "kpi": {Driver: [변수...]},
      "aptitude": {Driver: [변수...]}
    }
    """
    raw = pd.read_excel(excel_path, sheet_name=SHEET_NAME)

    # ① 인사기록카드: 제목 행 찾고, 그 아래 헤더/데이터 위치를 고정(시트 구조 기준)
    row_hr_title = find_section_row(raw, "①")
    # 실제 파일 기준으로:
    # - 헤더 행: row_hr_title + 1 (여기에 '구분' / '기본 인적사항' ... )
    # - 데이터 시작: row_hr_title + 2 (Trust부터)
    hr_map = parse_matrix_table(raw, header_row=row_hr_title + 1, start_row=row_hr_title + 2)

    # ② KPI
    row_kpi_title = find_section_row(raw, "②")
    # KPI는 헤더가 'KPI 항목' 행, 데이터가 그 아래
    kpi_map = parse_matrix_table(raw, header_row=row_kpi_title + 1, start_row=row_kpi_title + 2)

    # ③ 인적성
    row_apt_title = find_section_row(raw, "③")
    apt_map = parse_matrix_table(raw, header_row=row_apt_title + 1, start_row=row_apt_title + 2)

    return {"hr_card": hr_map, "kpi": kpi_map, "aptitude": apt_map}


# -------------------------
# 3) 직원 데이터 -> 7 드라이버 점수 산출 (No-ML)
# -------------------------
def compute_driver_scores_for_employee(
    emp_row: pd.Series,
    mapping: Dict[str, Dict[str, List[str]]],
    source_weights: Dict[str, float] = None,
) -> Dict[str, Any]:
    """
    emp_row: 직원 1명의 데이터 row (컬럼명=변수명)
    mapping: 위에서 만든 {"hr_card":..., "kpi":..., "aptitude":...}
    source_weights: 각 소스별 가중치 (합 1 권장)
    """
    if source_weights is None:
        source_weights = {"hr_card": 0.3, "kpi": 0.5, "aptitude": 0.2}

    # 드라이버별로 소스 점수 -> 가중합
    out = {}

    for drv in DRIVERS:
        per_source_scores = {}
        for source, drv_map in mapping.items():
            vars_list = drv_map.get(drv, [])
            vals = []
            for v in vars_list:
                if v in emp_row.index:
                    vals.append(normalize_1_to_5(emp_row[v]))
            per_source_scores[source] = safe_mean(vals)

        # 가중합 (값이 없는 소스는 자동 제외하고 재정규화)
        usable = {s: sc for s, sc in per_source_scores.items() if sc is not None}
        if not usable:
            out[drv] = None
            continue

        w = {s: source_weights.get(s, 0.0) for s in usable}
        w = {k: v for k, v in w.items() if v > 0}
        w = w if w else {s: 1.0 for s in usable}
        w = {k: v / sum(w.values()) for k, v in w.items()}

        score = sum(usable[s] * w[s] for s in usable)
        out[drv] = round(float(score), 3)

    return {
        "driver_scores": out,
        "source_debug": {
            "source_weights": source_weights,
            # 필요하면 디버그로 per_source_scores도 넣을 수 있음
        },
    }

def build_driver_score_table(
    employee_df: pd.DataFrame,
    mapping: Dict[str, Dict[str, List[str]]],
    employee_id_col: str = "employee_id",
) -> pd.DataFrame:
    results = []
    for _, row in employee_df.iterrows():
        r = compute_driver_scores_for_employee(row, mapping)
        rec = {employee_id_col: row[employee_id_col]} if employee_id_col in employee_df.columns else {}
        rec.update(r["driver_scores"])
        results.append(rec)
    return pd.DataFrame(results)


# -------------------------
# 4) (선택) 텍스트 임베딩은 "설명 근거 검색"용으로만 저장 (ML 아님)
#    - 여기서는 구조만 제시 (실행하려면 openai 설치/키 필요)
# -------------------------
def build_text_embedding_vector_db(text_df: pd.DataFrame, text_cols: List[str]) -> pd.DataFrame:
    """
    MVP에서는 벡터 DB까지 안 만들어도 됨.
    README용이라면: '텍스트를 벡터화해 유사 케이스 검색/RAG에 활용'이라고만 써도 충분.
    실제 구현은 Step2/Step3 붙일 때 진행 추천.
    """
    text_df = text_df.copy()
    text_df["combined_text"] = text_df[text_cols].fillna("").agg("\n".join, axis=1)
    return text_df[[c for c in ["employee_id", "combined_text"] if c in text_df.columns]]


# -------------------------
# 5) 실행 예시 (직원 데이터는 너희 파일로 교체)
# -------------------------
if __name__ == "__main__":
    mapping = load_driver_variable_mapping(EXCEL_PATH)

    print("=== Parsed Mapping (샘플) ===")
    for src, m in mapping.items():
        print(f"\n[{src}]")
        for k in DRIVERS:
            print(k, "->", m.get(k, [])[:5], ("..." if len(m.get(k, [])) > 5 else ""))

    # ✅ 여기 employee_df는 너희 실제 정량 데이터(인사기록+KPI+인적성)로 교체하면 됨
    # 예시: CSV/DB에서 불러와서 컬럼명이 매핑 테이블의 변수명과 동일해야 함
#employee_df = pd.read_csv("your_employee_numeric_data.csv")
    
driver_scores_df = build_driver_score_table(employee_df, mapping, employee_id_col="employee_id")
driver_scores_df.to_csv("step1_driver_scores.csv", index=False)

import numpy as np
import pandas as pd

def make_demo_employee_df_from_mapping(mapping, n=50):
    """
    맥킨지 매핑 엑셀에 등장한 변수들을 기반으로
    50명짜리 가짜 직원 정량 데이터 생성
    (각 변수 값은 1~5)
    """
    vars_set = set()

    for src in mapping:
        for drv in mapping[src]:
            for v in mapping[src][drv]:
                # '●(반대)' 같은 표식 제거
                if "●" in v:
                    continue
                vars_set.add(v)

    vars_list = sorted(vars_set)

    df = pd.DataFrame({
        "employee_id": [f"E{i:03d}" for i in range(1, n + 1)]
    })

    for col in vars_list:
        df[col] = np.random.randint(1, 6, size=n)

    return df


# 🔥 50명 데모 직원 데이터 생성
employee_df = make_demo_employee_df_from_mapping(mapping, n=50)

employee_df.head()


driver_scores_df = build_driver_score_table(
    employee_df,
    mapping,
    employee_id_col="employee_id"
)

driver_scores_df.head()


=== Parsed Mapping (샘플) ===

[hr_card]
Trust -> ['공정성 인식', '규칙 준수', '근속 안정성', '신뢰 관련 평가', '윤리/신뢰 교육'] ...
Communication -> ['전공 일관성', '조직 간 협업 구조', '커뮤니케이션 교육', '커뮤니케이션/어학', '협업 부서 이동'] ...
Innovative Thinking -> ['다양한 직무 경험', '신기술·신자격', '신설/혁신 조직 이동', '전공 다양성', '혁신 성과 반영'] ...
Decision Making -> ['리더 포지션 이동', '성과급 구조', '의사결정 관련 자격', '의사결정 교육', '의사결정 권한 직위'] ...
Psychological Safety -> ['심리/리더십 교육', '안정성', '안정적 조직 경험', '안정적 조직 배치', '잦은 이동 여부'] ...
Conflict Management -> ['갈등 많은 조직 경험', '갈등 평가 기록', '갈등관리 교육', '갈등성 이동 여부', '중재·협상 자격'] 
Role Definition -> ['역할 몰입도', '역할 변화 적합성', '역할 성과 일치', '전공-직무 적합', '직무 전문 교육'] ...

[kpi]
Trust -> ['신뢰', '신뢰 기반 자율', '신뢰도', '신뢰성', '책임 이행'] ...
Communication -> ['소통 태도', '핵심 KPI', '협업 품질', '협업 효율'] 
Innovative Thinking -> ['개선 시도', '대안 다양성', '변화 태도', '핵심 지표'] 
Decision Making -> ['결정 속도', '의사결정 효율', '책임 수용', '판단 정확성', '판단 태도'] ...
Psychological Safety -> ['안전감'] 
Conflict Management -> ['갈등 태도', '갈등 해결'] 
Role Definition -> ['역할 수행력', '역할 정확성', '역할 집중도',

,employee_id,Trust,Communication,Innovative Thinking,Decision Making,Psychological Safety,Conflict Management,Role Definition
0,E001,1.887,1.529,1.586,1.605,1.711,1.464,1.974
1,E002,3.008,2.452,2.061,1.663,1.530,1.356,1.596
2,E003,3.019,1.305,1.764,1.780,1.648,1.355,2.222
3,E004,2.226,1.615,1.870,1.717,3.272,1.313,1.300
4,E005,1.923,1.806,1.939,1.842,3.267,1.154,1.992


In [14]:
import numpy as np
import pandas as pd
from typing import Dict, List, Optional, Any

# =========================
# 0) 설정
# =========================
EXCEL_PATH = "맥킨지 모델 적용 정량 데이터 변수 리스트.xlsx"
SHEET_NAME = "맥킨지X정량데이터 매핑 테이블"

DRIVERS = [
    "Trust",
    "Communication",
    "Innovative Thinking",
    "Decision Making",
    "Psychological Safety",
    "Conflict Management",
    "Role Definition",
]

SCORE_MIN, SCORE_MAX = 1.0, 5.0


# =========================
# 1) 유틸
# =========================
def is_blank(x) -> bool:
    return (pd.isna(x)) or (str(x).strip() == "")

def clean_cell(x) -> str:
    if pd.isna(x):
        return ""
    return str(x).strip()

def normalize_1_to_5(x) -> float:
    if x is None or pd.isna(x):
        return np.nan
    try:
        x = float(x)
    except:
        return np.nan

    if 0.0 <= x <= 1.0:
        return 1.0 + 4.0 * x
    if 0.0 <= x <= 100.0:
        return 1.0 + 4.0 * (x / 100.0)
    return min(max(x, SCORE_MIN), SCORE_MAX)

def safe_mean(values: List[float]) -> Optional[float]:
    vals = [v for v in values if v is not None and not pd.isna(v)]
    if not vals:
        return None
    return float(np.mean(vals))


# =========================
# 2) 매핑 테이블 파싱
# =========================
def find_section_row(df: pd.DataFrame, keyword: str) -> Optional[int]:
    for i in range(len(df)):
        row_text = " ".join([clean_cell(v) for v in df.iloc[i].tolist()])
        if keyword in row_text:
            return i
    return None

def parse_matrix_table(
    df: pd.DataFrame,
    header_row: int,
    start_row: int,
    driver_col_idx: int = 1
) -> Dict[str, List[str]]:
    headers = [clean_cell(x) for x in df.iloc[header_row].tolist()]
    out: Dict[str, List[str]] = {}

    for r in range(start_row, len(df)):
        driver = clean_cell(df.iloc[r, driver_col_idx])
        if is_blank(driver):
            break
        if driver not in DRIVERS:
            continue

        vars_for_driver = []
        for c in range(len(headers)):
            if c == driver_col_idx:
                continue

            cell = clean_cell(df.iloc[r, c])
            if cell in ["-", "–", "—", ""]:
                continue

            # ●는 "해당" 표시일 뿐 실제 컬럼명이 아님 -> 헤더를 변수명으로 사용
            if cell == "●":
                var_name = headers[c]
            else:
                var_name = cell

            if var_name and var_name not in ["구분", "KPI 항목", "인적성 영역"]:
                vars_for_driver.append(var_name)

        out[driver] = sorted(list(set(vars_for_driver)))

    return out

def load_driver_variable_mapping(excel_path: str) -> Dict[str, Dict[str, List[str]]]:
    raw = pd.read_excel(excel_path, sheet_name=SHEET_NAME)

    row_hr_title = find_section_row(raw, "①")
    row_kpi_title = find_section_row(raw, "②")
    row_apt_title = find_section_row(raw, "③")

    if row_hr_title is None or row_kpi_title is None or row_apt_title is None:
        raise ValueError("①/②/③ 섹션을 엑셀에서 찾지 못했어. 시트 구조/기호를 확인해줘.")

    hr_map = parse_matrix_table(raw, header_row=row_hr_title + 1, start_row=row_hr_title + 2)
    kpi_map = parse_matrix_table(raw, header_row=row_kpi_title + 1, start_row=row_kpi_title + 2)
    apt_map = parse_matrix_table(raw, header_row=row_apt_title + 1, start_row=row_apt_title + 2)

    return {"hr_card": hr_map, "kpi": kpi_map, "aptitude": apt_map}


# =========================
# 3) 점수 산출 (No-ML)
# =========================
def compute_driver_scores_for_employee(
    emp_row: pd.Series,
    mapping: Dict[str, Dict[str, List[str]]],
    source_weights: Dict[str, float] = None,
) -> Dict[str, Any]:
    if source_weights is None:
        source_weights = {"hr_card": 0.3, "kpi": 0.5, "aptitude": 0.2}

    out = {}

    for drv in DRIVERS:
        per_source_scores = {}

        for source, drv_map in mapping.items():
            vars_list = drv_map.get(drv, [])
            vals = []
            for v in vars_list:
                if v in emp_row.index:
                    vals.append(normalize_1_to_5(emp_row[v]))
            per_source_scores[source] = safe_mean(vals)

        usable = {s: sc for s, sc in per_source_scores.items() if sc is not None}
        if not usable:
            out[drv] = None
            continue

        w = {s: source_weights.get(s, 0.0) for s in usable}
        w = {k: v for k, v in w.items() if v > 0}
        if not w:
            w = {s: 1.0 for s in usable}
        ssum = sum(w.values())
        w = {k: v / ssum for k, v in w.items()}

        score = sum(usable[s] * w[s] for s in usable)
        out[drv] = round(float(score), 3)

    return out

def build_driver_score_table(
    employee_df: pd.DataFrame,
    mapping: Dict[str, Dict[str, List[str]]],
    employee_id_col: str = "employee_id",
) -> pd.DataFrame:
    results = []
    for _, row in employee_df.iterrows():
        rec = {}
        if employee_id_col in employee_df.columns:
            rec[employee_id_col] = row[employee_id_col]
        rec.update(compute_driver_scores_for_employee(row, mapping))
        results.append(rec)
    return pd.DataFrame(results)


# =========================
# 4) 50명 샘플 데이터 생성 (엑셀 매핑 기반)
# =========================
def make_demo_employee_df_from_mapping(mapping, n=50):
    vars_set = set()
    for src in mapping:
        for drv in mapping[src]:
            for v in mapping[src][drv]:
                if "●" in v:
                    continue
                vars_set.add(v)

    vars_list = sorted(vars_set)

    df = pd.DataFrame({"employee_id": [f"E{i:03d}" for i in range(1, n + 1)]})
    for col in vars_list:
        df[col] = np.random.randint(1, 6, size=n)
    return df


# =========================
# 5) 실행
# =========================
mapping = load_driver_variable_mapping(EXCEL_PATH)

employee_df = make_demo_employee_df_from_mapping(mapping, n=50)
driver_scores_df = build_driver_score_table(employee_df, mapping, employee_id_col="employee_id")

# ✅ 전체 50명 확인: head() 말고 shape / sample / tail
print("employee_df shape:", employee_df.shape)
print("driver_scores_df shape:", driver_scores_df.shape)

display(driver_scores_df)   # 주피터에서는 display가 전체를 '스크롤'로 보여줌
# 또는 필요하면 아래처럼
# print(driver_scores_df.to_string(index=False))

# 저장
employee_df.to_csv("demo_employee_raw_50.csv", index=False, encoding="utf-8-sig")
driver_scores_df.to_csv("demo_driver_scores_50.csv", index=False, encoding="utf-8-sig")

print("저장 완료: demo_employee_raw_50.csv, demo_driver_scores_50.csv")


employee_df shape: (50, 95)
driver_scores_df shape: (50, 8)


,employee_id,Trust,Communication,Innovative Thinking,Decision Making,Psychological Safety,Conflict Management,Role Definition
0,E001,1.909,1.293,1.715,1.507,1.558,2.347,1.818
1,E002,1.603,2.025,2.390,1.932,3.636,2.475,1.630
2,E003,1.908,2.478,1.591,2.501,3.551,1.974,2.217
3,E004,1.865,1.996,1.438,1.624,2.015,1.919,1.837
4,E005,2.037,1.830,3.179,1.753,1.402,1.287,1.640
5,E006,1.260,1.293,2.342,1.521,1.362,2.334,1.755
6,E007,1.911,2.161,1.915,1.871,1.732,3.455,1.890
7,E008,2.067,2.355,1.779,1.456,1.804,1.884,1.737
8,E009,1.899,2.154,2.046,2.644,1.590,2.850,1.912
9,E010,2.230,1.295,2.212,1.883,1.353,1.371,1.293


저장 완료: demo_employee_raw_50.csv, demo_driver_scores_50.csv
